<a href="https://colab.research.google.com/github/mikewwalker-gh/topic-finder-aid/blob/main/03_vectorstores_and_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[**bold text**!pip show tensorflow-text](https://)

In [1]:
!pip show tensorflow-text

Name: tensorflow-text
Version: 2.13.0
Summary: TF.Text is a TensorFlow library of text related ops, modules, and subgraphs.
Home-page: http://github.com/tensorflow/text
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: tensorflow, tensorflow-hub
Required-by: 


In [2]:
# !pip install tensorflow-text

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# /content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf

In [5]:
!pip show langchain

Name: langchain
Version: 0.0.273
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [6]:
# !pip install langchain

In [7]:
# !pip install pypdf

In [8]:
# !pip install typing-extensions --upgrade

In [9]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf"),


docs = []
for loader in loaders:
    docs.extend(loader.load())

In [10]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [11]:
splits = text_splitter.split_documents(docs)

In [12]:
len(splits)

57

In [13]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings()

In [14]:
import langchain
from langchain.embeddings import TensorflowHubEmbeddings

# The URL of the TensorFlow Hub embedding model to use.
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"

# Create an instance of the TensorflowHubEmbeddings class.
embedding = TensorflowHubEmbeddings(model_url=model_url)

In [15]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [16]:
embedding1 = embedding.embed_query(sentence1)

In [17]:
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [18]:
import numpy as np

In [19]:
np.dot(embedding1, embedding2), np.dot(embedding1, embedding3), np.dot(embedding2, embedding3)

(0.8770532130006156, 0.1959314032974775, 0.15008649710999022)

In [20]:
# !pip install chromadb

In [21]:
from langchain.vectorstores import Chroma

In [22]:
persist_directory = 'docs/chroma/'

In [ ]:
# !rm -rf ./docs/chroma  # remove old database files if any

In [23]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [24]:
print(vectordb._collection.count())

57


In [25]:
question = "what did they say about matlab?"

In [26]:
docs = vectordb.similarity_search(question,k=5)

In [30]:
type(docs), docs[0]

(list,
 Document(page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it\'s free, and for the purposes of  thi

In [31]:
docs[1]

Document(page_content='into his office and he said, "Oh, professo r, professor, thank you so much for your \nmachine learning class. I learned so much from it. There\'s this stuff that I learned in your \nclass, and I now use every day. And it\'s help ed me make lots of money, and here\'s a \npicture of my big house."  \nSo my friend was very excited. He said, "W ow. That\'s great. I\'m glad to hear this \nmachine learning stuff was actually useful. So what was it that you learned? Was it \nlogistic regression? Was it the PCA? Was it the data ne tworks? What was it that you \nlearned that was so helpful?" And the student said, "Oh, it was the MATLAB."  \nSo for those of you that don\'t know MATLAB yet, I hope you do learn it. It\'s not hard, \nand we\'ll actually have a short MATLAB tutori al in one of the discussion sections for \nthose of you that don\'t know it.  \nOkay. The very last piece of logistical th ing is the discussion s ections. So discussion \nsections will be taught by 

In [32]:
question = "what did they say about regression in the first lecture?"

In [33]:
docs = vectordb.similarity_search(question,k=5)

In [34]:
for doc in docs:
  print(doc.metadata)

{'page': 12, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 8, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 8, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 4, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 6, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}


In [35]:
question = "is there an email i can ask for help"

In [36]:
docs = vectordb.similarity_search(question, k=3)

In [39]:
for doc in docs:
  print(doc.metadata)

{'page': 5, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 5, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}
{'page': 4, 'source': '/content/drive/MyDrive/Colab Notebooks/machinelearning-lecture01.pdf'}


In [41]:
for i in range(len(docs)):
  print(i, docs[i])

0 page_content="cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to succeed and \ndo well in this class and even help you to enjoy this cla ss more is if you form a study \ngroup.  \nSo start looking around where you' re sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly enc

In [42]:
vectordb.persist()